# Redes Neurais Artificiais 2020.1 -- Projeto Prático 3.4

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:

## Reservado para a importação de bibliotecas

import pandas as pd
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

In [ ]:

# Leitura do dataset covtype.csv

df = pd.read_csv('./covtype.csv', sep=',')  
                                                
#eliminando todos os atributos categóricos('Wilderness_Area' e 'Soil_Type'), geramos o 'df_sac' (dataframe sem atributos categóricos)
for i in range(4):
    df_sac = df.drop([('Wilderness_Area' + str(i+1))], axis = 1)
for i in range(40):
    df_sac = df.drop([('Soil_Type' + str(i+1))], axis = 1)

# X = atributos preditores 
x = df_sac.drop(['Cover_Type'],axis = 1)
# Y = atributo alvo
y = df_sac['Cover_Type']

In [ ]:
# vetor top 3 configs das arquiteturas proprias
arquiteturas = []

# preenchimento do vetor com as 3 melhores configuracoes da arquitetura proprias
arquiteturas.append({                                                          
    "camadasOcultas": 1,"funcaoAtivacao": "relu", "hiperparametro": "sgd",
    "epocas": 100, "disposicaoNeuronios": 11, "acuracia": 0.75436,
    "fscore":0.57250
})
arquiteturas.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 100, "disposicaoNeuronios": 6, "acuracia": 0.74752,
    "fscore":0.51983
})
arquiteturas.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "disposicaoNeuronios": 4, "acuracia": 0.73415,
    "fscore":0.46785
})


In [1]:
#configs da busca em grade                                                              
scoring = {                                                                       
    'f1_score': make_scorer(f1_score, average="macro")
}

parametros ={                                                               
    "solver": ["sgd", "adam"] ,
    "batch_size": ['auto']  ,
    "learning_rate_init": [0.001, 0.001] ,
    "n_iter_no_change": [45, 55]  ,
    "max_iter": [250, 400] 
} 
c = 0
for i in arquiteturas:    
    classifier = MLPClassifier(hidden_layer_sizes=(i["disposicaoNeuronios"]),            
                            activation=i["funcaoAtivacao"], 
                            solver=i["hiperparametro"], 
                            max_iter=i["epocas"], 
                            verbose=False)
    
    clf = GridSearchCV(classifier, tuned_parameters, scoring=scoring, refit="f1_score") 
    clf.fit(x, y)                                                                                                  

NameError: ignored

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html
2. Treine o modelo com todos os dados
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas